In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
print(f'Originalny folder: {os.getcwd()}')
#os.chdir('c:/Projekty/sim/PARETO_5/input/dicom')
print(os.getcwd())

Originalny folder: /home/szmurlor/src/bdcalc/notebooks
/home/szmurlor/src/bdcalc/notebooks


Tutaj trzeba ustawić folder gdzie znajduje się cały bdcalc (w linii komend trzebaby było zrobić `export PYTHONPATH=.`)

In [3]:
# os.chdir('c:/Projekty/bdcalc/src')
os.chdir('/home/szmurlor/src/bdcalc/src')
print(os.getcwd())

/home/szmurlor/src/bdcalc/src


In [13]:
# Tutaj tylko sprawdzam czy nic nie zwiodło i jesteśmy w folderze, w którym dostępny jest moduł 'Beamlets'
import bd4cnn.bd4cnn
if "Beamlets" in dir (bd4cnn.bd4cnn):
    print("Current working directory configuration OK.")
else:
    print(f"ERROR! Current working directory configuration wrong: {os.getcwd()}")
    

Current working directory configuration OK.


Tutaj trzeba podać przypadek, który chcemy przeliczyć.

In [5]:
#PATIENT_FOLDER="c:/Projekty/sim/PARETO_5"
PATIENT_FOLDER="/doses-nfs/sim/1419704"
PATIENT_FOLDER="/doses-nfs/sim/527029"

In [14]:
args = lambda x: x
args.root_folder=PATIENT_FOLDER
bd4cnn.bd4cnn.do_run(args)

M
INFO:bdcalc:Reading contours for blizna from DICOM
dz spacing: 0.30000000000000004 
dz spacing: 0.30000000000000004 
dz spacing: 0.30000000000000004 
2020-12-30 17:42:41,316 - bdcalc - INFO - Reading contours for watroba-fragment from DICOM
INFO:bdcalc:Reading contours for watroba-fragment from DICOM
2020-12-30 17:42:41,337 - bdcalc - INFO - Reading contours for suma pluc from DICOM
INFO:bdcalc:Reading contours for suma pluc from DICOM
dz spacing: 0.30000000000000004 
dz spacing: 0.30000000000000004 
2020-12-30 17:42:41,660 - bdcalc - INFO - Reading contours for zz_srodek from DICOM
INFO:bdcalc:Reading contours for zz_srodek from DICOM
2020-12-30 17:42:41,849 - bdcalc - INFO - Reading contours for zz_ptv from DICOM
INFO:bdcalc:Reading contours for zz_ptv from DICOM
dz spacing: 0.30000000000000004 
dz spacing: 0.30000000000000004 
2020-12-30 17:42:42,088 - bdcalc - INFO - Reading contours for Ipsilat.lung from DICOM
INFO:bdcalc:Reading contours for Ipsilat.lung from DICOM
2020-12-30 1

In [21]:
import bdfileutils
r = bdfileutils.read_ndarray(f'{PATIENT_FOLDER}/output/roi_marks.nparray', dtype=np.int64)
ct = bdfileutils.read_ndarray(f'{PATIENT_FOLDER}/output/approximated_ct.nparray', dtype=np.float32)
doses = bdfileutils.read_ndarray(f'{PATIENT_FOLDER}/output/total_doses.nparray', dtype=np.float32)

INFO:bdfileutils:Reading ndarray data from file: /doses-nfs/sim/527029/output/roi_marks.nparray
INFO:bdfileutils:The ndarray data has shape: [146, 124, 248]
INFO:bdfileutils:Reading ndarray data from file: /doses-nfs/sim/527029/output/approximated_ct.nparray
INFO:bdfileutils:The ndarray data has shape: [146, 124, 248]
INFO:bdfileutils:Reading ndarray data from file: /doses-nfs/sim/527029/output/total_doses.nparray
INFO:bdfileutils:The ndarray data has shape: [146, 124, 248]


In [17]:
import matplotlib.cm as cm
from ipywidgets import interact
@interact(s=(0,ct.shape[0]-1))
def f(s):
    plt.imshow(ct[s,:,:], cmap=cm.gray)

interactive(children=(IntSlider(value=72, description='s', max=145), Output()), _dom_classes=('widget-interact…

In [18]:
roi_names = {}
with open(f'{PATIENT_FOLDER}/output/roi_mapping.txt') as f:
    for line in f:
        line = line.strip()
        # print(line.split(":"))
        cols = line.split(":")
        roi_names[cols[0]] = int(cols[1])
print(roi_names)        

{'Patient Outline': 1, 'wezly all PTV': 2, 'wezly all': 4, 'ptv': 8, 'ctv': 16, 'blizna PTV': 32, 'blizna': 64, 'watroba-fragment': 128, 'suma pluc': 256, 'zz_srodek': 512, 'zz_ptv': 1024, 'Ipsilat.lung': 2048, 'serce': 4096, 'przelyk': 8192, 'pluco P': 16384, 'pluco L': 32768, 'kanal kreg.+5mm': 65536, 'kanal kreg.': 131072, 'gl.kosci ram.': 262144, 'CouchSurface': 524288, 'CouchInterior': 1048576, 'zz_zdrowa piers': 2097152, 'zz_zdrowe': 4194304, 'zz_obwarz.': 8388608, 'Contralat.lung': 16777216, 'naczynia wienc.': 33554432}


In [37]:
from ipywidgets import interact
@interact(s=(0,r.shape[0]-1), roi=roi_names)
def d(s, roi):
    plt.subplots(1,1,dpi=50, figsize=(16,9))
    rp = np.copy(r[s, :, :])
    sel = (np.bitwise_and(rp,roi)) == roi
    rp[ sel ] = 1
    rp[ np.bitwise_not(sel) ] = 0
    plt.imshow(ct[s,:,:], cmap=cm.gray)
    plt.imshow( rp, alpha=0.55)

interactive(children=(IntSlider(value=72, description='s', max=145), Dropdown(description='roi', options={'Pat…

In [46]:
from ipywidgets import interact
@interact(s=(0,r.shape[0]-1), x=(0,r.shape[1]-1), y=(0,r.shape[2]-1), dmin=(0, np.max(doses)), dmax=(0, np.max(doses)), roi=roi_names)
def d(s, x, y, roi, dmin=0, dmax=np.max(doses)):
    f, [ax1, ax2, ax3] = plt.subplots(1,3,dpi=50, figsize=(16,9))
    d = np.copy(doses[s, :, :])
    d[d < dmin] = 0
    d[d > dmax] = 0
    ax1.imshow(ct[s,:,:], cmap=cm.gray)
    ax1.imshow(d, alpha=0.55, cmap=cm.jet)

    d = np.copy(doses[:, x, :])
    d[d < dmin] = 0
    d[d > dmax] = 0
    ax2.imshow(ct[:,x,:], cmap=cm.gray, origin='lower')
    ax2.imshow(d, alpha=0.55, cmap=cm.jet, origin='lower')

    d = np.copy(doses[:, :, y])
    d[d < dmin] = 0
    d[d > dmax] = 0
    ax3.imshow(ct[:,:,y], cmap=cm.gray, origin='lower')
    ax3.imshow(d, alpha=0.55, cmap=cm.jet, origin='lower')


interactive(children=(IntSlider(value=72, description='s', max=145), IntSlider(value=61, description='x', max=…